# Import packages

In [1]:
import pandas as pd
import numpy as np
import torch
import pandas as pd
from pathlib import Path

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from chemprop import data, featurizers, models, nn, uncertainty
from chemprop.models import save_model, load_model
from chemprop.cli.conf import NOW
from chemprop.cli.predict import find_models

%load_ext autoreload
%autoreload 2

# Training

# Change data inputs here

In [2]:
chemprop_dir = Path.cwd().parent
input_path = (
    chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv"
)  # path to your data .csv file
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
smiles_column = "smiles"  # name of the column containing SMILES strings
target_columns = ["lipo"]  # list of names of the columns containing targets

## Load data

In [3]:
df_input = pd.read_csv(input_path)
df_input

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


# Get SMILES and targets

In [4]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

In [ ]:
smis[:5]  # show first 5 SMILES strings

array(['Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14',
       'COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)CCc3ccccc23',
       'COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl',
       'OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(Cl)sc4[nH]3',
       'Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)NCC#N)c1'],
      dtype=object)

In [6]:
ys[:5]  # show first 5 targets

array([[ 3.54],
       [-1.18],
       [ 3.69],
       [ 3.37],
       [ 3.1 ]])

## Get molecule datapoints

In [7]:
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

## Perform data splitting for training, validation, and testing

In [8]:
# available split types
list(data.SplitType.keys())

['CV_NO_VAL',
 'CV',
 'SCAFFOLD_BALANCED',
 'RANDOM_WITH_REPEATED_SMILES',
 'RANDOM',
 'KENNARD_STONE',
 'KMEANS']

In [9]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

## Get MoleculeDataset

In [10]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

## Get DataLoader

In [11]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message Passing
A `Message passing` constructs molecular graphs using message passing to learn node-level hidden representations.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [12]:
mp = nn.BondMessagePassing()

## Aggregation
An `Aggregation` is responsible for constructing a graph-level representation from the set of node-level representations after message passing.

Available options can be found in ` nn.agg.AggregationRegistry`, including
- `agg = nn.MeanAggregation()`
- `agg = nn.SumAggregation()`
- `agg = nn.NormAggregation()`

In [13]:
print(nn.agg.AggregationRegistry)

ClassRegistry {
    'mean': <class 'chemprop.nn.agg.MeanAggregation'>,
    'sum': <class 'chemprop.nn.agg.SumAggregation'>,
    'norm': <class 'chemprop.nn.agg.NormAggregation'>
}


In [14]:
agg = nn.MeanAggregation()

## Feed-Forward Network (FFN)

A `FFN` takes the aggregated representations and make target predictions.

Available options can be found in `nn.PredictorRegistry`.

For regression:
- `ffn = nn.RegressionFFN()`
- `ffn = nn.MveFFN()`
- `ffn = nn.EvidentialFFN()`

For classification:
- `ffn = nn.BinaryClassificationFFN()`
- `ffn = nn.BinaryDirichletFFN()`
- `ffn = nn.MulticlassClassificationFFN()`
- `ffn = nn.MulticlassDirichletFFN()`

For spectral:
- `ffn = nn.SpectralFFN()` # will be available in future version

In [ ]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [16]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [17]:
# Change to other predictor if needed.
ffn = nn.MveFFN(output_transform=output_transform)

## Batch Norm
A `Batch Norm` normalizes the outputs of the aggregation by re-centering and re-scaling.

Whether to use batch norm

In [18]:
batch_norm = False

## Metrics
`Metrics` are the ways to evaluate the performance of model predictions.

Available options can be found in `metrics.MetricRegistry`, including

In [19]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mae': <class 'chemprop.nn.metrics.MAEMetric'>,
    'mse': <class 'chemprop.nn.metrics.MSEMetric'>,
    'rmse': <class 'chemprop.nn.metrics.RMSEMetric'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAEMetric'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSEMetric'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSEMetric'>,
    'r2': <class 'chemprop.nn.metrics.R2Metric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROCMetric'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRCMetric'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracyMetric'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Metric'>,
    'bce': <class 'chemprop.nn.metrics.BCEMetric'>,
    'ce': <class 'chemprop.nn.metrics.CrossEntropyMetric'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'sid': <class 'chemprop.nn.metrics.SIDMetric'>,
    'wasserstein': <class '

In [20]:
metric_list = [
    nn.metrics.RMSEMetric(),
    nn.metrics.MAEMetric(),
]  # Only the first metric is used for training and early stopping

## Constructs MPNN

In [ ]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): Identity()
  (predictor): MveFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=2, bias=True)
      )
    )
    (criterion): MVELoss(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metrics): ModuleList(
    (0): RMSEMetric(task_weights=[[1.0]])
    (1): MAEMetric(task_weights=[[1.0]])
    (2): MVELoss(task_weights=[[1.0]])
  )
)

# Set up trainer

In [22]:
model_output_dir = Path(f"chemprop_training/{NOW}")
monitor_mode = "min" if mpnn.metrics[0].minimize else "max"
checkpointing = ModelCheckpoint(
    model_output_dir / "checkpoints",
    "best-{epoch}-{val_loss:.2f}",
    "val_loss",
    mode=monitor_mode,
    save_last=True,
)

In [23]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    callbacks=[checkpointing],
    devices=1,
    max_epochs=50,  # number of epochs to train for
)

/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/scli/miniconda3/envs/chemprop_v2/lib/python3.1 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Start training

In [24]:
trainer.fit(mpnn, train_loader, val_loader)

/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/scli/miniconda3/envs/chemprop_v2/lib/python3.1 ...
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loading `train_dataloader` to estimate number of stepping batches.
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 36.88it/s, train_loss=1.130, val_loss=1.200]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 32.71it/s, train_loss=1.130, val_loss=1.200]



# Model prediction

In [25]:
results = trainer.test(mpnn, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 225.71it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  batch_averaged_test/mae  │     0.769349217414856     │
│ batch_averaged_test/rmse  │    0.9241093397140503     │
└───────────────────────────┴───────────────────────────┘

# Save the best model

In [26]:
best_model_path = checkpointing.best_model_path
model = mpnn.__class__.load_from_checkpoint(best_model_path)
p_model = model_output_dir / "best.pt"
save_model(p_model, model)

# Predicting

# Change model input here

In [27]:
model_paths = find_models([model_output_dir])
model_paths

[PosixPath('chemprop_training/2024-10-19T01-49-29/best.pt')]

In [28]:
chemprop_dir = Path.cwd().parent
test_path = chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv"
df_test = pd.read_csv(test_path)
test_dset = data.MoleculeDataset(test_data, featurizer)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)
df_test

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


In [29]:
# use the validation set from the training as the calibration set as an example
cal_dset = data.MoleculeDataset(val_data, featurizer)
cal_loader = data.build_dataloader(cal_dset, num_workers=num_workers, shuffle=False)

# Uncertainty predictor
A uncertianty predictor can make model predictions and associated uncertainty predictions.

Available options can be found in `uncertainty.PredictorRegistry`.

In [ ]:
print(uncertainty.UncertaintyPredictorRegistry)

ClassRegistry {
    'nouncertaintypredictor': <class 'chemprop.uncertainty.predictor.NoUncertaintyPredictor'>,
    'mve': <class 'chemprop.uncertainty.predictor.MVEPredictor'>,
    'ensemble': <class 'chemprop.uncertainty.predictor.EnsemblePredictor'>,
    'classification': <class 'chemprop.uncertainty.predictor.ClassPredictor'>,
    'evidential-total': <class 'chemprop.uncertainty.predictor.EvidentialTotalPredictor'>,
    'evidential-epistemic': <class 'chemprop.uncertainty.predictor.EvidentialEpistemicPredictor'>,
    'evidential-aleatoric': <class 'chemprop.uncertainty.predictor.EvidentialAleatoricPredictor'>,
    'dropout': <class 'chemprop.uncertainty.predictor.DropoutPredictor'>,
    'spectra-roundrobin': <class 'chemprop.uncertainty.predictor.RoundRobinSpectraPredictor'>,
    'classification-dirichlet': <class 'chemprop.uncertainty.predictor.ClassificationDirichletPredictor'>,
    'multiclass-dirichlet': <class 'chemprop.uncertainty.predictor.MulticlassDirichletPredictor'>,
    

In [31]:
unc_predictor = uncertainty.MVEPredictor()

# Uncertainty calibrator
A uncertianty calibrator can calibrate the predicted uncertainties.

Available options can be found in `uncertainty.UncertaintyCalibratorRegistry`.

For regression:
- ZScalingCalibrator
- ZelikmanCalibrator
- MVEWeightingCalibrator
- RegressionConformalCalibrator

For binary classification:
- PlattCalibrator
- IsotonicCalibrator
- MultilabelConformalCalibrator

For multiclass classification:
- MulticlassConformalCalibrator
- AdaptiveMulticlassConformalCalibrator
- IsotonicMulticlassCalibrator

In [ ]:
print(uncertainty.UncertaintyCalibratorRegistry)

ClassRegistry {
    'zscaling': <class 'chemprop.uncertainty.calibrator.ZScalingCalibrator'>,
    'zelikman-interval': <class 'chemprop.uncertainty.calibrator.ZelikmanCalibrator'>,
    'mve-weighting': <class 'chemprop.uncertainty.calibrator.MVEWeightingCalibrator'>,
    'conformal-regression': <class 'chemprop.uncertainty.calibrator.RegressionConformalCalibrator'>,
    'platt': <class 'chemprop.uncertainty.calibrator.PlattCalibrator'>,
    'isotonic': <class 'chemprop.uncertainty.calibrator.IsotonicCalibrator'>,
    'conformal-multilabel': <class 'chemprop.uncertainty.calibrator.MultilabelConformalCalibrator'>,
    'conformal-multiclass': <class 'chemprop.uncertainty.calibrator.MulticlassConformalCalibrator'>,
    'conformal-adaptive': <class 'chemprop.uncertainty.calibrator.AdaptiveMulticlassConformalCalibrator'>,
    'isotonic-multiclass': <class 'chemprop.uncertainty.calibrator.IsotonicMulticlassCalibrator'>
}


In [33]:
unc_calibrator = uncertainty.ZScalingCalibrator()

# Uncertainty evaluator
A uncertianty evaluator can evaluates the quality of uncertainty estimates.

Available options can be found in `uncertainty.UncertaintyEvaluatorRegistry`.

For regression:
- NLLRegressionEvaluator
- CalibrationAreaEvaluator
- ExpectedNormalizedErrorEvaluator
- SpearmanEvaluator
- RegressionConformalEvaluator

For binary classification:
- NLLClassEvaluator
- MultilabelConformalEvaluator

For multiclass classification:
- NLLMulticlassEvaluator
- MulticlassConformalEvaluator

In [ ]:
print(uncertainty.UncertaintyEvaluatorRegistry)

ClassRegistry {
    'nll-regression': <class 'chemprop.uncertainty.evaluator.NLLRegressionEvaluator'>,
    'miscalibration_area': <class 'chemprop.uncertainty.evaluator.CalibrationAreaEvaluator'>,
    'ence': <class 'chemprop.uncertainty.evaluator.ExpectedNormalizedErrorEvaluator'>,
    'spearman': <class 'chemprop.uncertainty.evaluator.SpearmanEvaluator'>,
    'conformal-coverage-regression': <class 'chemprop.uncertainty.evaluator.RegressionConformalEvaluator'>,
    'nll-classification': <class 'chemprop.uncertainty.evaluator.NLLClassEvaluator'>,
    'conformal-coverage-classification': <class 'chemprop.uncertainty.evaluator.MultilabelConformalEvaluator'>,
    'nll-multiclass': <class 'chemprop.uncertainty.evaluator.NLLMulticlassEvaluator'>,
    'conformal-coverage-multiclass': <class 'chemprop.uncertainty.evaluator.MulticlassConformalEvaluator'>
}


In [35]:
unc_evaluators = [
    uncertainty.NLLRegressionEvaluator(),
    uncertainty.CalibrationAreaEvaluator(),
    uncertainty.ExpectedNormalizedErrorEvaluator(),
    uncertainty.SpearmanEvaluator(),
]

## Load model

In [36]:
models = [load_model(model_path, multicomponent=False) for model_path in model_paths]

# Setup trainer

In [ ]:
trainer = pl.Trainer(logger=False, enable_progress_bar=True, accelerator="cpu", devices=1)

/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/scli/miniconda3/envs/chemprop_v2/lib/python3.1 ...
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`

# Make uncertainty estimations

In [38]:
test_predss, test_uncss = unc_predictor(test_loader, models, trainer)
test_preds = test_predss.mean(0)
test_uncs = test_uncss.mean(0)

df_test = pd.DataFrame(
    {
        "smiles": test_dset.smiles,
        "target": test_dset.Y.reshape(-1),
        "pred": test_preds.reshape(-1),
        "unc": test_uncs.reshape(-1),
    }
)

df_test

/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 115.86it/s]

,smiles,target,pred,unc
0,Cc1ccc(NC(=O)c2cscn2)cc1-n1cnc2ccc(N3CCN(C)CC3...,2.06,2.699275,1.048715
1,O=C(Nc1nnc(C(=O)Nc2ccc(N3CCOCC3)cc2)o1)c1ccc(C...,1.92,2.219236,1.274951
2,CNCCCC12CCC(c3ccccc31)c1ccccc12,0.89,2.825118,0.984752
3,Oc1ncnc2scc(-c3ccsc3)c12,2.25,3.052753,0.892364
4,C=CC(=O)Nc1cccc(CN2C(=O)N(c3c(Cl)c(OC)cc(OC)c3...,2.04,2.170408,1.350023
5,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCCC1,3.13,2.800348,1.014192
6,O=C(COc1ccccc1)c1ccccc1,2.87,2.287262,1.236441
7,CC(C)c1ccc2oc3nc(N)c(C(=O)O)cc3c(=O)c2c1,1.10,2.386894,1.209242
8,N#Cc1ccc(F)c(-c2cc(C(F)(F)F)ccc2OCC(=O)O)c1,-0.16,1.484101,1.762928
9,COc1cnc(-c2ccccn2)nc1N(C)C,1.90,2.880277,0.986577


# Apply uncertainty calibration

In [39]:
cal_predss, cal_uncss = unc_predictor(cal_loader, models, trainer)
average_cal_preds = cal_predss.mean(0)
average_cal_uncs = cal_uncss.mean(0)
cal_targets = cal_dset.Y
cal_mask = torch.from_numpy(np.isfinite(cal_targets))
cal_targets = np.nan_to_num(cal_targets, nan=0.0)
cal_targets = torch.from_numpy(cal_targets)
unc_calibrator.fit(average_cal_preds, average_cal_uncs, cal_targets, cal_mask)

cal_test_uncs = unc_calibrator.apply(test_uncs)
df_test["cal_unc"] = cal_test_uncs
df_test

/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/scli/miniconda3/envs/chemprop_v2/lib/python3.1 ...
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 124.64it/s]


/home/scli/packages/chemprop/chemprop/uncertainty/calibrator.py:107: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  self.scalings = torch.tensor(scalings)


,smiles,target,pred,unc,cal_unc
0,Cc1ccc(NC(=O)c2cscn2)cc1-n1cnc2ccc(N3CCN(C)CC3...,2.06,2.699275,1.048715,0.915769
1,O=C(Nc1nnc(C(=O)Nc2ccc(N3CCOCC3)cc2)o1)c1ccc(C...,1.92,2.219236,1.274951,1.113325
2,CNCCCC12CCC(c3ccccc31)c1ccccc12,0.89,2.825118,0.984752,0.859915
3,Oc1ncnc2scc(-c3ccsc3)c12,2.25,3.052753,0.892364,0.779239
4,C=CC(=O)Nc1cccc(CN2C(=O)N(c3c(Cl)c(OC)cc(OC)c3...,2.04,2.170408,1.350023,1.178880
5,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCCC1,3.13,2.800348,1.014192,0.885623
6,O=C(COc1ccccc1)c1ccccc1,2.87,2.287262,1.236441,1.079697
7,CC(C)c1ccc2oc3nc(N)c(C(=O)O)cc3c(=O)c2c1,1.10,2.386894,1.209242,1.055946
8,N#Cc1ccc(F)c(-c2cc(C(F)(F)F)ccc2OCC(=O)O)c1,-0.16,1.484101,1.762928,1.539441
9,COc1cnc(-c2ccccn2)nc1N(C)C,1.90,2.880277,0.986577,0.861509


# Uncertainty evaluation

In [ ]:
test_targets = test_dset.Y
test_mask = torch.from_numpy(np.isfinite(test_targets))
test_targets = np.nan_to_num(test_targets, nan=0.0)
test_targets = torch.from_numpy(test_targets)

for evaluator in unc_evaluators:
    evaluation = evaluator.evaluate(test_preds, cal_test_uncs, test_targets, test_mask)
    print(f"{evaluator.alias}: {evaluation.tolist()}")

nll-regression: [1.4527006621180294]
miscalibration_area: [0.05119999125599861]
ence: [0.48281348026478793]
spearman: [-0.333333283662796]


/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028
